### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option('display.precision', 10)
CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross ,  cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)
# print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-co

In [5]:
df_reverse ,  cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse,params=conf.HDBSCAN_PARAMS_REVERSE)
print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean        ymid_mean
16   2024-04-28 02:11:27  2024-04-28 02:45:42      6          15  1398.5833333333   336.0000000000
51   2024-04-28 03:07:23  2024-04-28 03:08:47      3          50  2064.6666666667   102.6666666667
52   2024-04-28 03:10:56  2024-04-28 03:13:10      3          51  2119.5000000000    46.1666666667
0    2024-04-28 03:17:35  2024-04-28 16:22:51     27          -1  1523.5740740741   367.5185185185
30   2024-04-28 03:24:17  2024-04-28 03:30:54      3          29  2046.8333333333    88.0000000000
53   2024-04-28 03:39:09  2024-04-28 03:40:58      2          52  2036.0000000000   129.7500000000
54   2024-04-28 03:44:31  2024-04-28 03:47:48      2          53  2105.5000000000    61.2500000000
67   2024-04-28 03:52:10  2024-04-28 03:52:21      2          66  2112.5000000000    63.0000000000
68   2024-04-28 03:54:13  2024-04-28 03:54:33      2          67  1983.7500000000   164.5000000000
55   2024-

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-co

result: The clustering now seems good enough with time_biased_distance4 metric (care more on time than coordinate and time diff is more than x amount it probably is not from the same group)

TRY assigning vehicle to cluster first ( then เก็บตก large cluster only?)

In [6]:
# tony = Cluster.assign_cross_cluster_to_vehicle_in_lifetime(df_vehicle,cluster_cross,distance_metric="euclidean")
# print(tony[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

In [7]:
tony = Cluster.assign_cross_cluster_to_vehicle_in_lifetime(df_vehicle,cluster_cross,distance_metric="euclidean")
# print(tony[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())
kenny = Cluster.assign_reverse_cluster_to_vehicle_in_lifetime(tony,cluster_reverse,distance_metric="euclidean")
print(kenny[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

                 camera       timestamp_precise         lifetime    xmid   ymid plate_number vehicle_type  count_cross  count_reverse    cluster_cross_list  cluster_reverse_list
2315  mbk-14-12-vehicle 2024-04-28 02:27:41.480    14.2359998226  1706.0  194.0       327722       pickup            0              0                    []                    []
2242  mbk-14-12-vehicle 2024-04-28 03:08:09.313    18.3550000191  1564.5  181.5      4กบ5887       pickup            0              0                    []                    []
2241  mbk-14-12-vehicle 2024-04-28 03:08:27.068    49.8659999371  1397.0  140.5      6กถ9907       pickup            0              0                    []                    []
2215  mbk-14-12-vehicle 2024-04-28 03:18:58.899    46.8269999027  1551.5  172.0          อก5       pickup            0              0                    []                    []
2204  mbk-14-12-vehicle 2024-04-28 03:21:31.980     2.6389999390  1401.5  174.0       ทห1197       pickup     


bad case : 06:47:56.476   516.1789999008  1747.0  200.0       340223       pickup      1714286876          1714287392      9        [87, 111, 151, 152]

TODO: find a better metric function to calculate distances